GRA2PES Demo
================

This notebook demonstrates multiple components of the GRA2PES framework, including:
- **Base data loading**: Loading the base data given the GRA2PES configuration.
- **Regridding**: Regridding the base LCC data to the target Lat/Lon grid. 
- **Regridded data loading**: Loading the regridded data given the GRA2PES configuration.
- **Basic processing**: Applying things like unit conversion etc. to the regridded data.
- **Emissions visualization**: Visualizing the regridded data using matplotlib.
- **Ratio visualization**: Visualizing the ratio of two regridded datasets.


### Setup
Ensure that the parameters in inventories/gra2pes/gra2pes_config.py are set correctly for your use case.

In [1]:
#Import necessary libraries
import os
import xarray as xr
import numpy as np
import pandas as pd
import xesmf as xe
import sys 
import matplotlib.pyplot as plt
import pyproj
import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
sys.path.append('..')
from inventories.gra2pes import gra2pes_utils , gra2pes_config, gra2pes_regrid
from utils import gen_utils, datetime_utils

# Base Data Loading/Examining
"Base" data referes to the data that is loaded from the dataset downloaded and organized by inventories/gra2pes/gra2pes_base_creator.py. These are the extracted tar.gz files from either the NOAA FTP or HTTPS servers. 

In [4]:
config = gra2pes_config.Gra2pesConfig() # Load the configuration 
extra_ids = 'methane' #define any "extra" ids that are not included in the default set of species
specs = ['CO2','CO','HC01']#,'HC02','HC14','NH3','NOX','SO2']

BGH = gra2pes_utils.BaseGra2pesHandler(config,specs = specs, extra_ids = extra_ids) #create the handler object

sector = 'AG'
year = 2021
month = 1
day_type = 'weekdy'
base_ds = BGH.load_fmt_fullday(sector, year, month, day_type, check_extra=False)
base_ds

<xarray.Dataset> Size: 8GB
Dimensions:   (utc_hour: 24, zlevel: 20, south_north: 1008, west_east: 1332)
Coordinates:
    XLAT      (south_north, west_east) float32 5MB 18.19 18.2 ... 52.42 52.41
    XLONG     (south_north, west_east) float32 5MB -121.8 -121.7 ... -58.58
  * utc_hour  (utc_hour) int64 192B 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
Dimensions without coordinates: zlevel, south_north, west_east
Data variables:
    CO2       (utc_hour, zlevel, south_north, west_east) float32 3GB dask.array<chunksize=(1, 1, 1008, 1332), meta=np.ndarray>
    CO        (utc_hour, zlevel, south_north, west_east) float32 3GB dask.array<chunksize=(1, 1, 1008, 1332), meta=np.ndarray>
    HC01      (utc_hour, zlevel, south_north, west_east) float32 3GB dask.array<chunksize=(1, 1, 1008, 1332), meta=np.ndarray>
Attributes: (12/20)
    CEN_LON:        -97.84192
    MAP_PROJ_CHAR:  Lambert Conformal
    TITLE:          GRA2PESv1.0 Emissions Dataset
    TRUELAT1:       33.0
    DX:             4000.0
    MOAD_CEN_LAT:   39.34594
    ...             ...
    CEN_LAT:        39.34594
    day_of_week:    weekday
    sector:         AG
    year:           2021
    month:          1
    day_type:       weekdy

# Regridding
This is an example of what happens in the gra2pes_regrid.py script to transform the base data from the LCC grid to the target Lat/Lon grid. Configuration for the regridding is set in inventories/gra2pes/gra2pes_config.py in the Gra2pesRegridConfig class.

In [6]:
# NOTE: This will take a bit of time to run (~3 minutes)
regrid_config = gra2pes_config.Gra2pesRegridConfig(config) # Load the regrid configuration
gra2pes_regridder = gra2pes_utils.Gra2pesRegridder(regrid_config) # Create the regridder object
base_ds = gra2pes_regrid.sum_on_dim(base_ds, dim ='zlevel') #Sum over the vertical levels to get a 2D dataset for much faster regridding
regridded_ds = gra2pes_regridder.regrid(base_ds) # Regrid the dataset to the desired grid
regridded_ds = gra2pes_regrid.slice_extent(regridded_ds,extent={'lon_min': -113, 'lon_max': -111, 'lat_min': 40, 'lat_max': 42}) # Slice the dataset to a specific extent
regridded_ds # Show the regridded dataset

ValueError: Dimensions ('zlevel',) not found in data dimensions ('utc_hour', 'south_north', 'west_east')

# Working with Regridded Data
This section demonstrates how to load the regridded and saved data from the gra2pes_regrid.py script. This allows the user to load the necessary data for further processing, visualization, or analysis.

In [2]:
config = gra2pes_config.Gra2pesConfig() # Load the configuration for the regridded dataset

# Define the regrid ID, which corresponds to the grid resolution. This is definde in the configuration file.
regrid_id = '0.025x0.025' 

# Define the date range for which you want to open the regridded dataset
dtr = datetime_utils.DateTimeRange('2021-01-01','2021-12-31 23:59',tz = 'UTC')

# Create the regridded handler object
rgh = gra2pes_utils.RegriddedGra2pesHandler(config,regrid_id) 

# Open the regridded dataset for the specified date range and sectors
regridded_ds = rgh.open_ds_inrange(dtr,sectors='all')

# Open the grid area dataset to get the cell area for the regridded grid
gca = xr.open_dataset(os.path.join(rgh.regridded_path,'details','grid_out_area.nc'))['cell_area']

In [4]:
# The regridded dataset is stored as an xarray dataset, separated by lat, lon, year, month, daytype (satdy, sundy, weekdy), 
# and sector as it is defined in the original GRA2PES tar files. 

regridded_ds # Show the regridded dataset

<xarray.Dataset> Size: 3GB
Dimensions:   (lat: 80, lon: 80, year: 1, month: 12, day_type: 3, utc_hour: 24,
               sector: 19)
Coordinates:
  * lat       (lat) float64 640B 40.02 40.05 40.07 40.1 ... 41.95 41.97 42.0
  * lon       (lon) float64 640B -113.0 -113.0 -112.9 ... -111.1 -111.0 -111.0
  * year      (year) int64 8B 2021
  * month     (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * day_type  (day_type) <U6 72B 'satdy' 'sundy' 'weekdy'
  * utc_hour  (utc_hour) int64 192B 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
  * sector    (sector) <U13 988B 'AG' 'AVIATION' 'COMM' ... 'WASTE' 'total'
Data variables:
    CO2       (lat, lon, year, month, day_type, utc_hour, sector) float32 420MB ...
    CO        (lat, lon, year, month, day_type, utc_hour, sector) float32 420MB ...
    HC01      (lat, lon, year, month, day_type, utc_hour, sector) float32 420MB ...
    HC02      (lat, lon, year, month, day_type, utc_hour, sector) float32 420MB ...
    HC14      (lat, lon, year, month, day_type, utc_hour, sector) float32 420MB ...
    NH3       (lat, lon, year, month, day_type, utc_hour, sector) float32 420MB ...
    NOX       (lat, lon, year, month, day_type, utc_hour, sector) float32 420MB ...
    SO2       (lat, lon, year, month, day_type, utc_hour, sector) float32 420MB ...
Attributes:
    TITLE:          GRA2PESv1.0 Emissions Dataset
    year:           2021
    regrid_method:  conservative
    git_hash:       dfcdc05d502c0daa3b701f4133d5e91c8994b0e0

We can also rework the time dimension to be more user-friendly and subsettable. This turns the multiple dimensions representing time (year, month, day_type, hour) into a single time dimension that can be easily indexed.

In [ ]:
# Rework the dataset to a more convenient format. This can take a bit of time to run (~3 minutes for full dataset)
ds = rgh.rework_ds_dt(regridded_ds)

In [7]:
#View the reworked dataset
ds

<xarray.Dataset> Size: 34GB
Dimensions:   (lat: 80, lon: 80, sector: 19, datetime: 8760)
Coordinates:
  * lat       (lat) float64 640B 40.02 40.05 40.07 40.1 ... 41.95 41.97 42.0
  * lon       (lon) float64 640B -113.0 -113.0 -112.9 ... -111.1 -111.0 -111.0
  * sector    (sector) <U13 988B 'AG' 'AVIATION' 'COMM' ... 'WASTE' 'total'
  * datetime  (datetime) datetime64[ns] 70kB 2021-01-01 ... 2021-12-31T23:00:00
Data variables:
    CO2       (lat, lon, sector, datetime) float32 4GB 0.0 0.0 ... 25.81 25.12
    CO        (lat, lon, sector, datetime) float32 4GB 0.0 0.0 ... 0.6175 0.6164
    HC01      (lat, lon, sector, datetime) float32 4GB 0.0 0.0 0.0 ... 1.61 1.62
    HC02      (lat, lon, sector, datetime) float32 4GB 0.0 0.0 ... 0.001219
    HC14      (lat, lon, sector, datetime) float32 4GB 0.0 0.0 ... 0.001941
    NH3       (lat, lon, sector, datetime) float32 4GB 0.0486 ... 0.00986
    NOX       (lat, lon, sector, datetime) float32 4GB 0.0 0.0 ... 0.04794
    SO2       (lat, lon, sector, datetime) float32 4GB 0.0 0.0 ... 0.0002331
Attributes:
    TITLE:          GRA2PESv1.0 Emissions Dataset
    year:           2021
    regrid_method:  conservative
    git_hash:       dfcdc05d502c0daa3b701f4133d5e91c8994b0e0